In [1]:
import tensorflow as tf
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
training_set = tf.keras.utils.image_dataset_from_directory(
    "train",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    
)

Found 71965 files belonging to 48 classes.


In [3]:
validation_set = tf.keras.utils.image_dataset_from_directory(
    "valid",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    
)

Found 18693 files belonging to 48 classes.


In [4]:
training_set

<BatchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 48), dtype=tf.float32, name=None))>

In [22]:
from tensorflow.keras import models, layers

data_augmentation = tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"), 
    layers.experimental.preprocessing.RandomRotation(0.2),
    layers.experimental.preprocessing.RandomTranslation(0.2,0.2),
    layers.experimental.preprocessing.RandomZoom(0.2),
    layers.experimental.preprocessing.Rescaling(1./255)
])

In [23]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout
from tensorflow.keras.models import Sequential

In [24]:
model = Sequential()

In [25]:
model.add(data_augmentation)

In [26]:
model.add(Conv2D(filters = 32, kernel_size=3, padding = "same", activation = "relu", input_shape=(128,128,3)))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))

In [27]:
model.add(Conv2D(filters = 64, kernel_size=3, padding = "same", activation = "relu"))
model.add(Conv2D(filters=64, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))

In [28]:
model.add(Conv2D(filters = 128, kernel_size=3, padding = "same", activation = "relu"))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))

In [29]:
model.add(Conv2D(filters = 256, kernel_size=3, padding = "same", activation = "relu"))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))

In [30]:
model.add(Conv2D(filters = 512, kernel_size=3, padding = "same", activation = "relu"))
model.add(Conv2D(filters=256, kernel_size=3, activation="relu"))
model.add(MaxPool2D(pool_size=2, strides=2))

In [31]:
model.add(Dropout(0.25))

In [32]:
model.add(Flatten())

In [33]:
model.add(Dense(units=1500, activation="relu"))

In [34]:
model.add(Dropout(0.4))

In [35]:
model.add(Dense(units=48, activation="softmax"))

In [36]:
model.build(input_shape=(None, 128,128,3))

In [37]:
model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.0001),loss="categorical_crossentropy", metrics = ["accuracy"])

In [38]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_2 (Sequential)   (None, 128, 128, 3)       0         
                                                                 
 conv2d_10 (Conv2D)          (None, 128, 128, 32)      896       
                                                                 
 conv2d_11 (Conv2D)          (None, 126, 126, 32)      9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 63, 63, 32)       0         
 2D)                                                             
                                                                 
 conv2d_12 (Conv2D)          (None, 63, 63, 64)        18496     
                                                                 
 conv2d_13 (Conv2D)          (None, 61, 61, 64)        36928     
                                                      

In [40]:
training_history = model.fit(x=training_set,validation_data=validation_set, epochs=1)

2249/2249 [==============================] - 2273s 1s/step - loss: 0.1818 - accuracy: 0.9399 - val_loss: 0.2204 - val_accuracy: 0.9265


In [41]:
train_loss, train_acc = model.evaluate(training_set)

2249/2249 [==============================] - 337s 150ms/step - loss: 0.1919 - accuracy: 0.9384


In [42]:
print(train_loss,train_acc)

0.19191978871822357 0.9383589029312134


In [43]:
val_loss,val_acc = model.evaluate(validation_set)

585/585 [==============================] - 88s 150ms/step - loss: 0.2204 - accuracy: 0.9265


In [44]:
print(val_loss, val_acc)

0.22041581571102142 0.9264965653419495


In [46]:
model.save("predict_DiseaseNew2.h5")

In [47]:
training_history.history

{'loss': [0.18184533715248108],
 'accuracy': [0.9398735761642456],
 'val_loss': [0.2204158455133438],
 'val_accuracy': [0.9264965653419495]}

In [49]:
class_name = validation_set.class_names
class_name

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy',
 'Blueberry___healthy',
 'Cherry_(including_sour)___Powdery_mildew',
 'Cherry_(including_sour)___healthy',
 'Cinnamon healthy',
 'Cinnamon leaf spot',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Corn_(maize)___Common_rust_',
 'Corn_(maize)___Northern_Leaf_Blight',
 'Corn_(maize)___healthy',
 'Grape___Black_rot',
 'Grape___Esca_(Black_Measles)',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Grape___healthy',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Peach___Bacterial_spot',
 'Peach___healthy',
 'Pepper,_bell___Bacterial_spot',
 'Pepper,_bell___healthy',
 'Potato___Early_blight',
 'Potato___Late_blight',
 'Potato___healthy',
 'Raspberry___healthy',
 'Soybean___healthy',
 'Squash___Powdery_mildew',
 'Strawberry___Leaf_scorch',
 'Strawberry___healthy',
 'Tea algal leaf',
 'Tea antrhacnose',
 'Tea bird eye spot',
 'Tea brown blight',
 'Tea gray light',
 'Tea healthy',
 'Tea red

In [50]:
test_set =  tf.keras.utils.image_dataset_from_directory(
    "valid",
    labels="inferred",
    label_mode="categorical",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(128, 128),
    shuffle=False,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
    
)

Found 18693 files belonging to 48 classes.


In [51]:
yPred = model.predict(test_set)
yPred, yPred.shape

585/585 [==============================] - 87s 149ms/step


(array([[1.0000000e+00, 2.1348834e-10, 1.1130041e-16, ..., 8.4814673e-20,
         6.6264062e-17, 1.8440719e-16],
        [1.0000000e+00, 1.5661195e-11, 6.5076108e-19, ..., 9.9061846e-23,
         9.8351144e-19, 5.1954098e-19],
        [1.0000000e+00, 2.4330140e-09, 5.5977348e-16, ..., 6.0150868e-20,
         9.5369670e-17, 2.0400505e-15],
        ...,
        [3.9117406e-08, 1.9946694e-12, 2.6233984e-09, ..., 1.4629629e-08,
         4.6861556e-11, 9.9956852e-01],
        [8.5698240e-08, 1.3330195e-13, 9.7242547e-10, ..., 4.1615220e-08,
         1.6098073e-12, 9.9992049e-01],
        [4.2031640e-13, 7.7275568e-21, 6.5430187e-15, ..., 4.5306539e-18,
         5.9908242e-16, 9.9999976e-01]], dtype=float32),
 (18693, 48))

In [52]:
predictedCategories = tf.argmax(yPred, axis=1)

In [53]:
predictedCategories

<tf.Tensor: shape=(18693,), dtype=int64, numpy=array([ 0,  0,  0, ..., 47, 47, 47], dtype=int64)>

In [54]:
trueCategories = tf.concat([y for x,y in test_set],axis = 0)
trueCategories

<tf.Tensor: shape=(18693, 48), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [55]:
yTrue = tf.argmax(trueCategories,axis=1)
yTrue

<tf.Tensor: shape=(18693,), dtype=int64, numpy=array([ 0,  0,  0, ..., 47, 47, 47], dtype=int64)>

In [56]:
from sklearn.metrics import classification_report

In [57]:
classification_report(yTrue, predictedCategories, target_names= class_name)

'                                                    precision    recall  f1-score   support\n\n                                Apple___Apple_scab       0.97      0.98      0.97       504\n                                 Apple___Black_rot       0.93      1.00      0.96       497\n                          Apple___Cedar_apple_rust       0.93      0.98      0.96       440\n                                   Apple___healthy       0.96      0.96      0.96       502\n                               Blueberry___healthy       0.98      0.90      0.94       454\n          Cherry_(including_sour)___Powdery_mildew       0.97      0.92      0.94       421\n                 Cherry_(including_sour)___healthy       0.99      0.94      0.96       456\n                                  Cinnamon healthy       0.92      0.88      0.90       131\n                                Cinnamon leaf spot       0.89      0.74      0.81        85\nCorn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.98      